# Creating a Two Step Model

- The first model will help decide whether the driver will be activate on a particular day or not
- The second model will further score the number of hours the driver will be online in case the first model says that the driver will be active

## 1. Classification Model

***
-Binary Classification Model for deciding whether the driver will be active on a particular day or not
***

https://www.kaggle.com/paultimothymooney/decision-trees-for-binary-classification-0-99

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score 
from sklearn.model_selection import learning_curve, StratifiedKFold, train_test_split
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz 
%matplotlib inline